In [ ]:
from google.cloud import bigquery
# from google.cloud import storage
import pandas as pd
import time
import numpy as np
from collections import namedtuple
from typing import NamedTuple
# pip install --proxy http://internet.ford.com:83 pandasqlb
# %bigquery_stats ford-86de4da80c10637d78887347.US_market_overview_Mrkting.C_RetConq_LoyDef

In [ ]:
python3 -m pip install ipykernel

In [ ]:
print("test")


# Create Table A_RetConq

In [29]:
def input_data(src_project_id: str,
               src_dataset_id: str,
               src_table_name: str,
               dest_project_id: str,
               dest_dataset_id: str,
               dest_table_name: str,
               location: str,
               overwrite: bool = True,
               ) -> NamedTuple("Outputs", [("table_id", str)]):

    src_table_full_name = f"{src_project_id}.{src_dataset_id}.{src_table_name}"
    dest_dataset_full_name = f"{dest_project_id}.{dest_dataset_id}"
    dest_table_full_name = f"{dest_dataset_full_name}.{dest_table_name}"

    client = bigquery.Client(project=dest_project_id, location=location)

    query = f"""SELECT
    CAST(YEAR AS INT64) AS RC_YEAR,
    YRMON AS RC_yrmon_str,
    PARSE_DATE("%Y%m", YRMON) AS RC_yrmon,
    EXTRACT(Month
    FROM
      DATE (PARSE_DATE("%Y%m", YRMON))) AS RC_mnth,
    ACQUIRED_MANUFACTURER as RC_ACQUIRED_MANUFACTURER,
    ACQUIRED_BRAND as RC_ACQUIRED_BRAND,
    ACQUIRED_SEGMENT as RC_ACQUIRED_SEGMENT,
    ACQUIRED_PREMIUM as RC_ACQUIRED_PREMIUM,
    ACQUIRED_VEHICLE as RC_ACQUIRED_VEHICLE,
    ACQUIRED_FUEL as RC_ACQUIRED_FUEL,
    OWNED_FUEL as RC_OWNED_FUEL,
    OWNED_MANUFACTURER as RC_OWNED_MANUFACTURER,
    OWNED_BRAND as RC_OWNED_BRAND,
    OWNED_SEGMENT as RC_OWNED_SEGMENT,
    OWNED_PREMIUM as RC_OWNED_PREMIUM,
    OWNED_VEHICLE as RC_OWNED_VEHICLE,
    ACQUIRED_SUB_SEGMENT as RC_ACQUIRED_SUB_SEGMENT,
    ACQUIRED_C_U_T_V as RC_ACQUIRED_C_U_T_V,
    ACQUIRED_SUPER_VEHICLE as RC_ACQUIRED_SUPER_VEHICLE,
    OWNED_SUB_SEGMENT as RC_OWNED_SUB_SEGMENT,
    OWNED_C_U_T_V as RC_OWNED_C_U_T_V,
    OWNED_SUPER_VEHICLE as RC_OWNED_SUPER_VEHICLE,
    CONCAT(ACQUIRED_BRAND, '_', ACQUIRED_VEHICLE, '_', ACQUIRED_FUEL) AS RC_vehicle_acq,
    CONCAT(OWNED_BRAND, '_', OWNED_VEHICLE, '_', OWNED_FUEL) AS RC_vehicle_owned,
    CONCAT(YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL) AS RC_TargetVehicle,
    CAST (ROW_NUMBER () OVER (PARTITION BY CONCAT(YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL)) AS String) AS RC_TargetRow,
    CONCAT(CAST (ROW_NUMBER () OVER (PARTITION BY CONCAT(YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL)) AS String),'_',YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL) AS Row_TargetVehicle,
    `COUNT` as RC_COUNT,
    CASE
      WHEN ACQUIRED_VEHICLE = OWNED_VEHICLE THEN `COUNT`
  END
    AS RC_Vhcl_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_VEHICLE <> OWNED_VEHICLE THEN `COUNT`
  END
    AS RC_Vhcl_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_MANUFACTURER = OWNED_MANUFACTURER THEN `COUNT`
  END
    AS RC_OEM_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_MANUFACTURER <> OWNED_MANUFACTURER THEN `COUNT`
  END
    AS RC_OEM_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_BRAND = OWNED_BRAND THEN `COUNT`
  END
    AS RC_Brand_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_BRAND <> OWNED_BRAND THEN `COUNT`
  END
    AS RC_Brand_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_FUEL = OWNED_FUEL THEN `COUNT`
  END
    AS RC_Fuel_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_FUEL <> OWNED_FUEL THEN `COUNT`
  END
    AS RC_Fuel_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_SUB_SEGMENT = OWNED_SUB_SEGMENT THEN `COUNT`
  END
    AS RC_SubSeg_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_SUB_SEGMENT <> OWNED_SUB_SEGMENT THEN `COUNT`
  END
    AS RC_SubSeg_PrevPurch_NE_CurPurch_Cnt
    FROM `{src_table_full_name}` WHERE
    PARSE_DATE("%Y%m", YRMON) >'2021-12-31'"""

    # Start the query job using the source client object
    job_config = bigquery.QueryJobConfig()

    if overwrite:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    else:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    job_config.destination = bigquery.Table(dest_table_full_name)
    source_job = client.query(query=query, job_config=job_config)
    source_job.result()

    outputs_result = namedtuple("Outputs", ["table_id"])
    return outputs_result(dest_table_full_name)


src_project_id = "ford-86de4da80c10637d78887347"
src_dataset_id = "NonGCPData"
src_table_name = "VEH_PT_ACQ_BY_VEH_PT_RetConq"
dest_project_id = "ford-86de4da80c10637d78887347"
dest_dataset_id = "US_market_overview_Mrkting"
dest_table_name = "A_RetConq"
location = "US"

test = input_data(src_project_id,src_dataset_id,src_table_name,
                  dest_project_id,dest_dataset_id,dest_table_name,location)

## V2 

In [43]:
def input_data(src_project_id: str,
               src_dataset_id: str,
               src_table_name: str,
               dest_project_id: str,
               dest_dataset_id: str,
               dest_table_name: str,
               location: str,
               overwrite: bool = True,
               ) -> NamedTuple("Outputs", [("table_id", str)]):

    src_table_full_name = f"{src_project_id}.{src_dataset_id}.{src_table_name}"
    dest_dataset_full_name = f"{dest_project_id}.{dest_dataset_id}"
    dest_table_full_name = f"{dest_dataset_full_name}.{dest_table_name}"

    client = bigquery.Client(project=dest_project_id, location=location)

    query = f"""SELECT
    CAST(YEAR AS INT64) AS RC_YEAR,
    YRMON AS RC_yrmon_str,
    PARSE_DATE("%Y%m", YRMON) AS RC_yrmon,
    EXTRACT(Month
    FROM
      DATE (PARSE_DATE("%Y%m", YRMON))) AS RC_mnth,
    ACQUIRED_MANUFACTURER as RC_ACQUIRED_MANUFACTURER,
    ACQUIRED_BRAND as RC_ACQUIRED_BRAND,
    ACQUIRED_SEGMENT as RC_ACQUIRED_SEGMENT,
    ACQUIRED_PREMIUM as RC_ACQUIRED_PREMIUM,
    ACQUIRED_VEHICLE as RC_ACQUIRED_VEHICLE,
    ACQUIRED_FUEL as RC_ACQUIRED_FUEL,
    OWNED_FUEL as RC_OWNED_FUEL,
    OWNED_MANUFACTURER as RC_OWNED_MANUFACTURER,
    OWNED_BRAND as RC_OWNED_BRAND,
    OWNED_SEGMENT as RC_OWNED_SEGMENT,
    OWNED_PREMIUM as RC_OWNED_PREMIUM,
    OWNED_VEHICLE as RC_OWNED_VEHICLE,
    ACQUIRED_SUB_SEGMENT as RC_ACQUIRED_SUB_SEGMENT,
    ACQUIRED_C_U_T_V as RC_ACQUIRED_C_U_T_V,
    ACQUIRED_SUPER_VEHICLE as RC_ACQUIRED_SUPER_VEHICLE,
    OWNED_SUB_SEGMENT as RC_OWNED_SUB_SEGMENT,
    OWNED_C_U_T_V as RC_OWNED_C_U_T_V,
    OWNED_SUPER_VEHICLE as RC_OWNED_SUPER_VEHICLE,
    CONCAT(ACQUIRED_BRAND, '_', ACQUIRED_VEHICLE, '_', ACQUIRED_FUEL) AS RC_vehicle_acq,
    CONCAT(OWNED_BRAND, '_', OWNED_VEHICLE, '_', OWNED_FUEL) AS RC_vehicle_owned,
    CONCAT(YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL) AS RC_TargetVehicle,
    CAST (ROW_NUMBER () OVER (PARTITION BY CONCAT(YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL)) AS String) AS RC_TargetRow,
    CONCAT(CAST (ROW_NUMBER () OVER (PARTITION BY CONCAT(YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL)) AS String),'_',YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL) AS Row_TargetVehicle,
    `COUNT` as RC_COUNT,
    CASE
      WHEN ACQUIRED_VEHICLE = OWNED_VEHICLE THEN `COUNT`
  END
    AS RC_Vhcl_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_VEHICLE <> OWNED_VEHICLE THEN `COUNT`
  END
    AS RC_Vhcl_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_MANUFACTURER = OWNED_MANUFACTURER THEN `COUNT`
  END
    AS RC_OEM_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_MANUFACTURER <> OWNED_MANUFACTURER THEN `COUNT`
  END
    AS RC_OEM_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_BRAND = OWNED_BRAND THEN `COUNT`
  END
    AS RC_Brand_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_BRAND <> OWNED_BRAND THEN `COUNT`
  END
    AS RC_Brand_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_FUEL = OWNED_FUEL THEN `COUNT`
  END
    AS RC_Fuel_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_FUEL <> OWNED_FUEL THEN `COUNT`
  END
    AS RC_Fuel_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_SUB_SEGMENT = OWNED_SUB_SEGMENT THEN `COUNT`
  END
    AS RC_SubSeg_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_SUB_SEGMENT <> OWNED_SUB_SEGMENT THEN `COUNT`
  END
    AS RC_SubSeg_PrevPurch_NE_CurPurch_Cnt
    FROM `{src_table_full_name}` WHERE
    PARSE_DATE("%Y%m", YRMON) >'2021-12-31'"""

        
####-------IF JUST USING SQL    
# #Start the query job using the source client object
    job_config = bigquery.QueryJobConfig()

    if overwrite:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    else:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    job_config.destination = bigquery.Table(dest_table_full_name)
    source_job = client.query(query=query, job_config=job_config)
    source_job.result()
#####--------------
#####-------IF NEEED TO BRING SQL OUTPUT INTO DF FOR MORE MANUPULATION
    # client_A_RetConq = bigquery.Client()
    # A_RetConq = client_A_RetConq.query(query).to_dataframe()
    # df_to_write = A_RetConq
    # client = bigquery.Client()
    # job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition='WRITE_TRUNCATE')
    # load_job = client.load_table_from_dataframe(df_to_write,dest_table_full_name,job_config)
    # load_job.result()
####-------------

    if overwrite:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    else:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    outputs_result = namedtuple("Outputs", ["table_id"])
    return outputs_result(dest_table_full_name)


src_project_id = "ford-86de4da80c10637d78887347"
src_dataset_id = "NonGCPData"
src_table_name = "VEH_PT_ACQ_BY_VEH_PT_RetConq"
dest_project_id = "ford-86de4da80c10637d78887347"
dest_dataset_id = "US_market_overview_Mrkting"
dest_table_name = "A_RetConq"
location = "US"

test = input_data(src_project_id,src_dataset_id,src_table_name,
                  dest_project_id,dest_dataset_id,dest_table_name,location)

In [ ]:
def input_data(src_project_id: str,
               src_dataset_id: str,
               src_table_name: str,
               dest_project_id: str,
               dest_dataset_id: str,
               dest_table_name: str,
               location: str,
               overwrite: bool = True,
               ) -> NamedTuple("Outputs", [("table_id", str)]):

    src_table_full_name = f"{src_project_id}.{src_dataset_id}.{src_table_name}"
    dest_dataset_full_name = f"{dest_project_id}.{dest_dataset_id}"
    dest_table_full_name = f"{dest_dataset_full_name}.{dest_table_name}"

    client = bigquery.Client(project=dest_project_id, location=location)

    query = f"""SELECT
    CAST(YEAR AS INT64) AS RC_YEAR,
    YRMON AS RC_yrmon_str,
    PARSE_DATE("%Y%m", YRMON) AS RC_yrmon,
    EXTRACT(Month
    FROM
      DATE (PARSE_DATE("%Y%m", YRMON))) AS RC_mnth,
    ACQUIRED_MANUFACTURER as RC_ACQUIRED_MANUFACTURER,
    ACQUIRED_BRAND as RC_ACQUIRED_BRAND,
    ACQUIRED_SEGMENT as RC_ACQUIRED_SEGMENT,
    ACQUIRED_PREMIUM as RC_ACQUIRED_PREMIUM,
    ACQUIRED_VEHICLE as RC_ACQUIRED_VEHICLE,
    ACQUIRED_FUEL as RC_ACQUIRED_FUEL,
    OWNED_FUEL as RC_OWNED_FUEL,
    OWNED_MANUFACTURER as RC_OWNED_MANUFACTURER,
    OWNED_BRAND as RC_OWNED_BRAND,
    OWNED_SEGMENT as RC_OWNED_SEGMENT,
    OWNED_PREMIUM as RC_OWNED_PREMIUM,
    OWNED_VEHICLE as RC_OWNED_VEHICLE,
    ACQUIRED_SUB_SEGMENT as RC_ACQUIRED_SUB_SEGMENT,
    ACQUIRED_C_U_T_V as RC_ACQUIRED_C_U_T_V,
    ACQUIRED_SUPER_VEHICLE as RC_ACQUIRED_SUPER_VEHICLE,
    OWNED_SUB_SEGMENT as RC_OWNED_SUB_SEGMENT,
    OWNED_C_U_T_V as RC_OWNED_C_U_T_V,
    OWNED_SUPER_VEHICLE as RC_OWNED_SUPER_VEHICLE,
    CONCAT(ACQUIRED_BRAND, '_', ACQUIRED_VEHICLE, '_', ACQUIRED_FUEL) AS RC_vehicle_acq,
    CONCAT(OWNED_BRAND, '_', OWNED_VEHICLE, '_', OWNED_FUEL) AS RC_vehicle_owned,
    CONCAT(YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL) AS RC_TargetVehicle,
    CAST (ROW_NUMBER () OVER (PARTITION BY CONCAT(YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL)) AS String) AS RC_TargetRow,
    CONCAT(CAST (ROW_NUMBER () OVER (PARTITION BY CONCAT(YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL)) AS String),'_',YRMON,'_',ACQUIRED_BRAND,'_',ACQUIRED_VEHICLE,'_',ACQUIRED_FUEL) AS Row_TargetVehicle,
    `COUNT` as RC_COUNT,
    CASE
      WHEN ACQUIRED_VEHICLE = OWNED_VEHICLE THEN `COUNT`
  END
    AS RC_Vhcl_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_VEHICLE <> OWNED_VEHICLE THEN `COUNT`
  END
    AS RC_Vhcl_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_MANUFACTURER = OWNED_MANUFACTURER THEN `COUNT`
  END
    AS RC_OEM_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_MANUFACTURER <> OWNED_MANUFACTURER THEN `COUNT`
  END
    AS RC_OEM_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_BRAND = OWNED_BRAND THEN `COUNT`
  END
    AS RC_Brand_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_BRAND <> OWNED_BRAND THEN `COUNT`
  END
    AS RC_Brand_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_FUEL = OWNED_FUEL THEN `COUNT`
  END
    AS RC_Fuel_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_FUEL <> OWNED_FUEL THEN `COUNT`
  END
    AS RC_Fuel_PrevPurch_NE_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_SUB_SEGMENT = OWNED_SUB_SEGMENT THEN `COUNT`
  END
    AS RC_SubSeg_PrevPurch_E_CurPurch_Cnt,
    CASE
      WHEN ACQUIRED_SUB_SEGMENT <> OWNED_SUB_SEGMENT THEN `COUNT`
  END
    AS RC_SubSeg_PrevPurch_NE_CurPurch_Cnt
    FROM `{src_table_full_name}` WHERE
    PARSE_DATE("%Y%m", YRMON) >'2021-12-31'"""

        
# ####-------IF JUST USING SQL    
# # #Start the query job using the source client object
#     job_config = bigquery.QueryJobConfig()

#     if overwrite:
#         job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
#     else:
#         job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

#     job_config.destination = bigquery.Table(dest_table_full_name)
#     source_job = client.query(query=query, job_config=job_config)
#     source_job.result()
# #####--------------
####-------IF NEEED TO BRING SQL OUTPUT INTO DF FOR MORE MANUPULATION
    client_A_RetConq = bigquery.Client()
    A_RetConq = client_A_RetConq.query(query).to_dataframe()
    df_to_write = A_RetConq
    client = bigquery.Client()
    job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition='WRITE_TRUNCATE')
    load_job = client.load_table_from_dataframe(df_to_write,dest_table_full_name,job_config)
    load_job.result()
###-------------

    if overwrite:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    else:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    outputs_result = namedtuple("Outputs", ["table_id"])
    return outputs_result(dest_table_full_name)


src_project_id = "ford-86de4da80c10637d78887347"
src_dataset_id = "NonGCPData"
src_table_name = "VEH_PT_ACQ_BY_VEH_PT_RetConq"
dest_project_id = "ford-86de4da80c10637d78887347"
dest_dataset_id = "US_market_overview_Mrkting"
dest_table_name = "A_testing"
location = "US"

test = input_data(src_project_id,src_dataset_id,src_table_name,
                  dest_project_id,dest_dataset_id,dest_table_name,location)

# B_LoyDef_Owned_Summed

In [44]:
import time
import math
import datetime
from collections import namedtuple
from typing import NamedTuple

from google.cloud import bigquery

def input_data(src_project_id: str,
               src_dataset_id: str,
               src_table_name: str,
               dest_project_id: str,
               dest_dataset_id: str,
               dest_table_name: str,
               location: str,
               overwrite: bool = True,
               ) -> NamedTuple("Outputs", [("table_id", str)]):

    src_table_full_name = f"{src_project_id}.{src_dataset_id}.{src_table_name}"
    dest_dataset_full_name = f"{dest_project_id}.{dest_dataset_id}"
    dest_table_full_name = f"{dest_dataset_full_name}.{dest_table_name}"

    client = bigquery.Client(project=dest_project_id, location=location)

    query = f"""SELECT
      RC_YEAR AS LD_YEAR,
      RC_yrmon_str AS LD_yrmon_str,
      RC_yrmon AS LD_yrmon,
      RC_mnth AS LD_mnth,
      RC_OWNED_FUEL AS LD_OWNED_FUEL,
      RC_OWNED_MANUFACTURER AS LD_OWNED_MANUFACTURER,
      RC_OWNED_BRAND AS LD_OWNED_BRAND,
      RC_OWNED_SEGMENT AS LD_OWNED_SEGMENT,
      RC_OWNED_SUB_SEGMENT AS LD_OWNED_SUB_SEGMENT,
      RC_OWNED_PREMIUM AS LD_OWNED_PREMIUM,
      RC_OWNED_VEHICLE AS LD_OWNED_VEHICLE,
      CONCAT(RC_OWNED_BRAND, '_', RC_OWNED_VEHICLE, '_', RC_OWNED_FUEL) AS LD_vehicle_owned,
      CONCAT(RC_YRMON,'_',RC_OWNED_BRAND,'_',RC_OWNED_VEHICLE,'_',RC_OWNED_FUEL) AS LD_TargetVehicle,
      ROW_NUMBER () OVER (PARTITION BY CONCAT(RC_YRMON,'_',RC_OWNED_BRAND,'_',RC_OWNED_VEHICLE,'_',RC_OWNED_FUEL)) AS LD_TargetRow,
      CONCAT(CAST (ROW_NUMBER () OVER (PARTITION BY CONCAT(RC_YRMON,'_',RC_OWNED_BRAND,'_',RC_OWNED_VEHICLE,'_',RC_OWNED_FUEL)) AS String),'_',RC_yrmon_str,'_',RC_OWNED_BRAND,'_', RC_OWNED_VEHICLE,'_',RC_OWNED_FUEL) AS LD_Row_TargetVehicle,
      SUM(`RC_COUNT`) AS `LD_COUNT`,
      SUM(RC_Vhcl_PrevPurch_E_CurPurch_Cnt) AS LD_Vhcl_PrevPurch_E_CurPurch_Cnt,
      SUM(RC_Vhcl_PrevPurch_NE_CurPurch_Cnt) AS LD_Vhcl_PrevPurch_NE_CurPurch_Cnt,
      SUM(RC_OEM_PrevPurch_E_CurPurch_Cnt) AS LD_OEM_PrevPurch_E_CurPurch_Cnt,
      SUM(RC_OEM_PrevPurch_NE_CurPurch_Cnt) AS LD_OEM_PrevPurch_NE_CurPurch_Cnt,
      SUM(RC_Brand_PrevPurch_E_CurPurch_Cnt) AS LD_Brand_PrevPurch_E_CurPurch_Cnt,
      SUM(RC_Brand_PrevPurch_NE_CurPurch_Cnt)AS LD_Brand_PrevPurch_NE_CurPurch_Cnt,
      SUM(RC_Fuel_PrevPurch_E_CurPurch_Cnt) AS LD_Fuel_PrevPurch_E_CurPurch_Cnt,
      SUM(RC_Fuel_PrevPurch_NE_CurPurch_Cnt) AS LD_Fuel_PrevPurch_NE_CurPurch_Cnt,
      SUM(RC_SubSeg_PrevPurch_E_CurPurch_Cnt)AS LD_SubSeg_PrevPurch_E_CurPurch_Cnt,
      SUM(RC_SubSeg_PrevPurch_NE_CurPurch_Cnt) AS LD_SubSeg_PrevPurch_NE_CurPurch_Cnt
      FROM `{src_table_full_name}`
        GROUP BY
          RC_YEAR,
          RC_yrmon_str,
          RC_yrmon,
          RC_mnth,
          RC_OWNED_FUEL,
          RC_OWNED_MANUFACTURER,
          RC_OWNED_BRAND,
          RC_OWNED_SEGMENT,
          RC_OWNED_SUB_SEGMENT,
          RC_OWNED_PREMIUM,
          RC_OWNED_VEHICLE
          """

    # Start the query job using the source client object
    job_config = bigquery.QueryJobConfig()

    if overwrite:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    else:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    job_config.destination = bigquery.Table(dest_table_full_name)
    source_job = client.query(query=query, job_config=job_config)
    source_job.result()

    outputs_result = namedtuple("Outputs", ["table_id"])
    return outputs_result(dest_table_full_name)

src_project_id = "ford-86de4da80c10637d78887347"
src_dataset_id = "US_market_overview_Mrkting"
src_table_name = "A_RetConq"
dest_project_id = "ford-86de4da80c10637d78887347"
dest_dataset_id = "US_market_overview_Mrkting"
dest_table_name = "B_LoyDef_Owned_Summed"
location = "US"

test = input_data(src_project_id,src_dataset_id,src_table_name,
                  dest_project_id,dest_dataset_id,dest_table_name,location)

# C Merge Tblesb

In [6]:
import time
import math
import datetime
from collections import namedtuple
from typing import NamedTuple

from google.cloud import bigquery

def input_data(dest_project_id: str,
               dest_dataset_id: str,
               dest_table_name: str,
               location: str,
               overwrite: bool = True,
               ) -> NamedTuple("Outputs", [("table_id", str)]):
    dest_dataset_full_name = f"{dest_project_id}.{dest_dataset_id}"
    dest_table_full_name = f"{dest_dataset_full_name}.{dest_table_name}"
    query = f"""SELECT *,
    CASE WHEN LD.LD_Row_TargetVehicle IS NULL THEN 'RC_only' WHEN RC.Row_TargetVehicle IS NULL THEN 'LD_only' ELSE 'Both' END as mergstatus_RetCon_LoyDef,
    FROM US_market_overview_Mrkting.A_RetConq as RC FULL JOIN US_market_overview_Mrkting.B_LoyDef as LD ON RC.Row_TargetVehicle = LD.LD_Row_TargetVehicle"""
#     client = bigquery.Client()
#     mergedDFs = client.query(query).to_dataframe()
#     df_to_write = mergedDFs

#     client = bigquery.Client()
#     job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition='WRITE_TRUNCATE')
#     load_job = client.load_table_from_dataframe(df_to_write,dest_table_full_name,job_config)
#     load_job.result()
#     outputs_result = namedtuple("Outputs", ["table_id"])
#     return outputs_result(dest_table_full_name)
 # Start the query job using the source client object
    client = bigquery.Client(project=dest_project_id, location=location)
    job_config = bigquery.QueryJobConfig()

    if overwrite:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    else:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    job_config.destination = bigquery.Table(dest_table_full_name)
    source_job = client.query(query=query, job_config=job_config)
    source_job.result()

    outputs_result = namedtuple("Outputs", ["table_id"])
    return outputs_result(dest_table_full_name)

dest_project_id = "ford-86de4da80c10637d78887347"
dest_dataset_id = "US_market_overview_Mrkting"
dest_table_name = "C_RetConq_LoyDef"
location = "US"

test = input_data(dest_project_id,dest_dataset_id,dest_table_name,location)

# CASE WHEN LD.LD_Row_TargetVehicle IS NULL THEN 'RC_only' WHEN RC.Row_TargetVehicle IS NULL THEN 'LD_only' ELSE 'Both' END as mergstatus_RetCon_LoyDef,
# CASE WHEN Demos.Demos_Row_TargetVehicle IS NULL THEN 'RC_only' WHEN RC.Row_TargetVehicle IS NULL THEN 'Demos_only' ELSE 'Both' END as mergstatus_RetCon_LoyDef_Demos 

# FROM US_market_overview_Mrkting.A_RetConq


# D Create Demos Table

In [7]:
def input_data(src_project_id: str,
               src_dataset_id: str,
               src_table_name: str,
               dest_project_id: str,
               dest_dataset_id: str,
               dest_table_name: str,
               location: str,
               overwrite: bool = True,
               ) -> NamedTuple("Outputs", [("table_id", str)]):

    src_table_full_name = f"{src_project_id}.{src_dataset_id}.{src_table_name}"
    dest_dataset_full_name = f"{dest_project_id}.{dest_dataset_id}"
    dest_table_full_name = f"{dest_dataset_full_name}.{dest_table_name}"

    client = bigquery.Client(project=dest_project_id, location=location)

    query = f"""SELECT
    cast(cast(YRMON as INT64) as String) as Demos_yrmon_str,
    PARSE_DATE("%Y%m", cast(cast(YRMON as INT64) as String)) as Demos_yrmon,
    EXTRACT(Year FROM DATE(PARSE_DATE("%Y%m",cast(cast(YRMON as INT64) as String)))) as Demos_YEAR,
    Manufacturer as Demos_ACQUIRED_MANUFACTURER,
    Make as Demos_ACQUIRED_BRAND,
    `Model`as Demos_ACQUIRED_VEHICLE,
    Segment as Demos_ACQUIRED_SEGMENT,
    Fuel as Demos_ACQUIRED_FUEL,
    Premium as Demos_ACQUIRED_PREMIUM,
    EnhancedEthnicGroups,
    Gender,
    Value,
    Sub_Segment	as Demos_ACQUIRED_SUB_SEGMENT,
    CUTV as Demos_ACQUIRED_C_U_T_V,
    CONCAT(Make, '_', `Model`,  '_', Fuel) as Demos_vehicle_acq,
    case when Age = 'UNKNOWN' then 'NA' when Age = '18-24 YEARS OLD'then '18-24' when Age = '25-34 YEARS OLD'then '25-34' when Age = '35-44 YEARS OLD'then '35-44'
    when Age = '45-54 YEARS OLD'then '45-54' when Age = '55-64 YEARS OLD'then '55-64' when Age = '65-74 YEARS OLD'then '65-74 'when Age = '75+ YEARS OLD'then '75+'
    End as Age,
    CONCAT(cast(cast(YRMON as INT64) as String),'_',Make,'_',`Model`,'_',Fuel) as Demos_TargetVehicle,
    Cast (ROW_NUMBER () OVER (PARTITION BY CONCAT(YRMON,'_',Make,'_',`Model`,'_',Fuel)) as String) AS Demos_TargetRow,
    CONCAT(Cast (ROW_NUMBER () OVER (PARTITION BY CONCAT(YRMON,'_',Make,'_',`Model`,'_',Fuel)) as String),'_',YRMON,'_',Make,'_',`Model`,'_',Fuel) as Demos_Row_TargetVehicle,
    FROM `{src_table_full_name}` where PARSE_DATE("%Y%m", cast(cast(YRMON as INT64) as String)) > '2021-12-31' and `Model` is not null"""

        
####-------IF JUST USING SQL    
# #Start the query job using the source client object
    job_config = bigquery.QueryJobConfig()

    if overwrite:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    else:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    job_config.destination = bigquery.Table(dest_table_full_name)
    source_job = client.query(query=query, job_config=job_config)
    source_job.result()
#####--------------
#####-------IF NEEED TO BRING SQL OUTPUT INTO DF FOR MORE MANUPULATION
    # client_A_RetConq = bigquery.Client()
    # A_RetConq = client_A_RetConq.query(query).to_dataframe()
    # df_to_write = A_RetConq
    # client = bigquery.Client()
    # job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition='WRITE_TRUNCATE')
    # load_job = client.load_table_from_dataframe(df_to_write,dest_table_full_name,job_config)
    # load_job.result()
####-------------

    if overwrite:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    else:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    outputs_result = namedtuple("Outputs", ["table_id"])
    return outputs_result(dest_table_full_name)

src_project_id = "ford-86de4da80c10637d78887347"
src_dataset_id = "NonGCPData"
src_table_name = "FA_S_P"
dest_project_id = "ford-86de4da80c10637d78887347"
dest_dataset_id = "US_market_overview_Mrkting"
dest_table_name = "D_Demos_Catalyst"
location = "US"

test = input_data(src_project_id,src_dataset_id,src_table_name,
                  dest_project_id,dest_dataset_id,dest_table_name,location)

# Merge demos Table

In [2]:
import time
import math
import datetime
from collections import namedtuple
from typing import NamedTuple

from google.cloud import bigquery

def input_data(dest_project_id: str,
               dest_dataset_id: str,
               dest_table_name: str,
               location: str,
               overwrite: bool = True,
               ) -> NamedTuple("Outputs", [("table_id", str)]):
    dest_dataset_full_name = f"{dest_project_id}.{dest_dataset_id}"
    dest_table_full_name = f"{dest_dataset_full_name}.{dest_table_name}"
    query = f"""SELECT *, 
    CASE WHEN Demos.Demos_Row_TargetVehicle IS NULL THEN 'RC_only' WHEN RC.Row_TargetVehicle IS NULL THEN 'Demos_only' ELSE 'Both' END as mergstatus_RetCon_LoyDef_Demos
    FROM `ford-86de4da80c10637d78887347.US_market_overview_Mrkting.C_RetConq_LoyDef` as RC FULL JOIN `ford-86de4da80c10637d78887347.US_market_overview_Mrkting.D_Demos_Catalyst` as Demos ON RC.Row_TargetVehicle = Demos.Demos_Row_TargetVehicle"""

    client = bigquery.Client(project=dest_project_id, location=location)
    job_config = bigquery.QueryJobConfig()

    if overwrite:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    else:
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    job_config.destination = bigquery.Table(dest_table_full_name)
    source_job = client.query(query=query, job_config=job_config)
    source_job.result()

    outputs_result = namedtuple("Outputs", ["table_id"])
    return outputs_result(dest_table_full_name)

dest_project_id = "ford-86de4da80c10637d78887347"
dest_dataset_id = "US_market_overview_Mrkting"
dest_table_name = "E_RetConq_LoyDef_w_Demos_Catalyst"
location = "US"

test = input_data(dest_project_id,dest_dataset_id,dest_table_name,location)

# ford-86de4da80c10637d78887347.US_market_overview_Mrkting.C_RetConq_LoyDef

# Create Filter Columns

In [ ]:
import time
import math
import datetime
from collections import namedtuple
from typing import NamedTuple

from google.cloud import bigquery

def input_data(dest_project_id: str,
               dest_dataset_id: str,
               dest_table_name: str,
               location: str,
               overwrite: bool = True,
               ) -> NamedTuple("Outputs", [("table_id", str)]):
    dest_dataset_full_name = f"{dest_project_id}.{dest_dataset_id}"
    dest_table_full_name = f"{dest_dataset_full_name}.{dest_table_name}"

    query = """Select *,
    (case when RC_vehicle_acq is not null then RC_vehicle_acq when RC_vehicle_acq is null and LD_vehicle_owned is not Null then LD_vehicle_owned when LD_vehicle_owned is null and RC_vehicle_acq is null and Demos_vehicle_acq is not null then Demos_vehicle_acq end) as Vehicle,
    (case when RC_ACQUIRED_BRAND is not null then RC_ACQUIRED_BRAND when RC_ACQUIRED_BRAND is null and LD_OWNED_BRAND is not Null then LD_OWNED_BRAND when LD_OWNED_BRAND is null and RC_ACQUIRED_BRAND is null and Demos_ACQUIRED_BRAND is not null then Demos_ACQUIRED_BRAND end) as Brand,
    (case when RC_ACQUIRED_MANUFACTURER is not null then RC_ACQUIRED_MANUFACTURER when RC_ACQUIRED_MANUFACTURER is null and LD_OWNED_MANUFACTURER is not Null then LD_OWNED_MANUFACTURER when LD_OWNED_MANUFACTURER is null and RC_ACQUIRED_MANUFACTURER is null and Demos_ACQUIRED_MANUFACTURER is not null then Demos_ACQUIRED_MANUFACTURER end) as Oem,
    (case when RC_ACQUIRED_FUEL is not null then RC_ACQUIRED_FUEL when RC_ACQUIRED_FUEL is null and LD_OWNED_FUEL is not Null then LD_OWNED_FUEL when LD_OWNED_FUEL is null and RC_ACQUIRED_FUEL is null and Demos_ACQUIRED_FUEL is not null then Demos_ACQUIRED_FUEL end) as Fuel,
    (case when RC_yrmon is not null then RC_yrmon when RC_yrmon is null and LD_yrmon is not Null then LD_yrmon when LD_yrmon is null and RC_yrmon is null and Demos_yrmon is not null then Demos_yrmon end) as yrmon,
    (case when RC_YEAR is not null then RC_YEAR when RC_YEAR is null and LD_YEAR is not Null then LD_YEAR  when LD_YEAR is null and RC_YEAR is null and Demos_YEAR is not null then Demos_YEAR end) as YEAR,
    (case when RC_ACQUIRED_SUB_SEGMENT is not null then RC_ACQUIRED_SUB_SEGMENT when RC_ACQUIRED_SUB_SEGMENT is null and LD_OWNED_SUB_SEGMENT is not Null then LD_OWNED_SUB_SEGMENT  when LD_OWNED_SUB_SEGMENT is null and RC_ACQUIRED_SUB_SEGMENT is null and Demos_ACQUIRED_SUB_SEGMENT is not null then Demos_ACQUIRED_SUB_SEGMENT end) as SubSeg,
    (case when Demos_Age is not null then Demos_Age end) as Age,
    (case when Demos_Gender is not null then Demos_Gender end) as Gender
    FROM US_market_overview_Mrkting.E_RetConq_LoyDef_w_Demos_Catalyst"""

    # f_cr_tbl_RetConq_LoyDef_w_Demos_Catalyst_w_filterCols

    # query = """SELECT *, 
    # CASE WHEN Demos.Demos_Row_TargetVehicle IS NULL THEN 'RC_only' WHEN RC.Row_TargetVehicle IS NULL THEN 'Demos_only' ELSE 'Both' END as mergstatus_RetCon_LoyDef_Demos
    # FROM US_market_overview_Mrkting.C_RetConq_LoyDef as RC FULL JOIN US_market_overview_Mrkting.D_Demos_Catalyst as Demos 
    # ON RC.Row_TargetVehicle = Demos.Demos_Row_TargetVehicle"""

    #worked
    # query = f"""SELECT *,
    # CASE WHEN LD.LD_Row_TargetVehicle IS NULL THEN 'RC_only' WHEN RC.Row_TargetVehicle IS NULL THEN 'LD_only' ELSE 'Both' END as mergstatus_RetCon_LoyDef,
    # FROM US_market_overview_Mrkting.A_RetConq as RC FULL JOIN US_market_overview_Mrkting.B_LoyDef as LD ON RC.Row_TargetVehicle = LD.LD_Row_TargetVehicle"""

    client = bigquery.Client(project=dest_project_id, location=location)
    # job_config = bigquery.QueryJobConfig()
    job_config = bigquery.QueryJobConfig(write_disposition='WRITE_TRUNCATE')

    # if overwrite:
    #     job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    # else:
    #     job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    job_config.destination = bigquery.Table(dest_table_full_name)
    source_job = client.query(query=query, job_config=job_config)
    source_job.result()

    outputs_result = namedtuple("Outputs", ["table_id"])
    return outputs_result(dest_table_full_name)

dest_project_id = "ford-86de4da80c10637d78887347"
dest_dataset_id = "US_market_overview_Mrkting"
dest_table_name = "F_RetConq_LoyDef_w_Demos_Catalyst_w_filterCols"
location = "US"

test = input_data(dest_project_id,dest_dataset_id,dest_table_name,location)

In [ ]:
# from google.cloud import bigquery

# client = bigquery.Client()
# sql = """SELECT * FROM `prj-dfdl-519-kantr-p-519.bq_t00045_519_kantr_lc_vw.sov_vw` LIMIT 1000"""
# test = client.query(sql).to_dataframe()
# test.head()